In [1]:
import os, gc, torch, PIL

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from utils.eval import save_iou_results
from utils.engine import xami_evaluate
from models.load import get_trained_model_from_setup
from utils.coco_eval import get_eval_params_dict
from data.dataset import ReflacxDataset, OurRadiologsitsDataset
from data.dataset import collate_fn
from utils.transforms import get_transform
from our_radiologist.load import get_anns

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
XAMI_MIMIC_PATH = "D:\XAMI-MIMIC"

In [3]:
gc.collect()
# torch.cuda.memory_summary(device=None, abbreviated=False)

use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
print(f"This notebook will running on device: [{device}]")

if use_gpu:
    torch.cuda.empty_cache()

This notebook will running on device: [cuda]


In [4]:
from models.load import ModelSetup

all_model_setups = [
    ModelSetup(
        name="original",
        use_clinical= False,
        use_custom_model=False,
        use_early_stop_model= True,
    ),
        ModelSetup(
        name="custom_without_clinical",
        use_clinical= False,
        use_custom_model=True,
        use_early_stop_model= True,
    ),
        ModelSetup(
        name="custom_with_clinical",
        use_clinical= True,
        use_custom_model=True,
        use_early_stop_model= True,
    )
]

In [5]:
model_setup = all_model_setups[0]

dataset_params_dict = {
    "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
    "with_clinical": model_setup.use_clinical,
    "dataset_mode": "unified",
    "bbox_to_mask": True,
    "labels_cols": [
        "Enlarged cardiac silhouette",
        "Atelectasis",
        "Pleural abnormality",
        "Consolidation",
        "Pulmonary edema",
        #  'Groundglass opacity', # 6th disease.
    ],
}

detect_eval_dataset = ReflacxDataset(
    **{**dataset_params_dict, "dataset_mode": "unified",},
    transforms=get_transform(train=False),
)

test_dataset = ReflacxDataset(
    **dataset_params_dict, split_str="test", transforms=get_transform(train=False),
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn,
)


In [6]:
model, model_path = get_trained_model_from_setup(
    dataset=detect_eval_dataset, setup=model_setup, device=device,
)

model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [7]:
normal_iou_thrs = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
normal_eval_params_dict = get_eval_params_dict(
    dataset=detect_eval_dataset, iou_thrs=normal_iou_thrs, use_iobb=True,
)



creating index...
index created!


In [8]:
test_evaluator = xami_evaluate(
        model,
        test_dataloader,
        device=device,
        params_dict=normal_eval_params_dict,
        # score_thres=score_thres,
    )

creating index...
index created!
Test:  [ 0/26]  eta: 0:01:32  model_time: 2.5516 (2.5516)  evaluator_time: 0.3898 (0.3898)  time: 3.5641  data: 0.5444  max mem: 2317
Test:  [25/26]  eta: 0:00:01  model_time: 0.3432 (0.4269)  evaluator_time: 0.3544 (0.3664)  time: 1.3013  data: 0.5361  max mem: 2391
Test: Total time: 0:00:37 (1.4301 s / it)
Averaged stats: model_time: 0.3432 (0.4269)  evaluator_time: 0.3544 (0.3664)
Accumulating evaluation results...
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.03s).
IoU metric: bbox
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets=100 ] = 0.148
 Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 10 ] = 0.248
 Average Precision  (AP) @[ IoBB=0.75      | area=   all | maxDets= 10 ] = 0.134
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area= small | maxDets= 10 ] = -1.000
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area=medium | maxDets= 10 ] = -1.000
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area= lar